In [6]:
from load_tools import load_model_and_data
from config import config, arch
# 主程序
# 1️⃣ 讀取模型與資料
checkpoint_filename = config["load_model_name"]
test_data = False # 測試模型準確度
model, train_dataloader, test_dataloader, images, labels = load_model_and_data(checkpoint_filename, test_data=test_data)

data_dir = "D://Paper/RGB_SFM/data/HeartCalcification/basic"  # 请替换为您的数据目录
save_dir = f'D://Paper/RGB_SFM/data/HeartCalcification/predict'


<class 'dataloader.HeartCalcification.HeartCalcificationColor'>
172
is train :True
augment_positive :True
need_augmentation :True
model data count :  9644
Label: -1, Count: 69446
Label: 0, Count: 4595
Label: 1, Count: 459
46
is train :False
augment_positive :True
need_augmentation :False
model data count :  1385
Label: -1, Count: 17355
Label: 0, Count: 1293
Label: 1, Count: 92
torch.Size([1385, 3, 45, 45]) torch.Size([1385, 2])
triangle
cReLU_percent
cReLU_percent
cReLU_percent
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
        RGB_Conv2d-1           [-1, 30, 45, 45]              90
          triangle-2           [-1, 30, 45, 45]               0
     cReLU_percent-3           [-1, 30, 45, 45]               0
               SFM-4           [-1, 30, 45, 45]               0
        RBF_Conv2d-5            [-1, 225, 9, 9]         168,750
     cReLU_percent-6            [-1, 225, 9, 9]               0
    

In [7]:
print(model)

RGB_SFMCNN_V2(
  (RGB_convs): Sequential(
    (0): Sequential(
      (0): RGB_Conv2d(initial = uniform, weight shape = torch.Size([30, 1, 1, 3]), cal_dist = LAB)
      (1): Sequential(
        (0): triangle(w = 1.6635164022445679)
        (1): cReLU_percent(percent=0.30000001192092896)
      )
      (2): SFM(filter=(1, 1), method=alpha_mean, alpha=[[1.]])
    )
    (1): Sequential(
      (0): RBF_Conv2d(initial = kaiming, weight shape = (225, 30, 5, 5))
      (1): Sequential(
        (0): cReLU_percent(percent=0.4000000059604645)
      )
      (2): SFM(
        filter=(3, 3), method=alpha_mean, alpha=[[1.         0.9875     0.975     ]
         [0.9625     0.95       0.9375    ]
         [0.92499995 0.91249996 0.9       ]]
      )
    )
    (2): Sequential(
      (0): RBF_Conv2d(initial = kaiming, weight shape = (625, 225, 1, 1))
      (1): Sequential(
        (0): cReLU_percent(percent=0.5)
      )
      (2): SFM(filter=(1, 1), method=alpha_mean, alpha=[[1.]])
    )
  )
  (fc1): Seque

In [8]:
from dataloader.heart_calcification.heart_calcification_results_display import *
from typing import List, Tuple, Any, Dict
from dataloader.heart_calcification.heart_calcification_data_processor import HeartCalcificationDataProcessor
import numpy as np
from dataloader.heart_calcification.image_split_data import ImageSplitData

from dataloader.heart_calcification.research_tool import *
from dataloader.heart_calcification.heart_calcification_results_display import *

def predict_2(model: Any, data_processor: HeartCalcificationDataProcessor) -> List[Tuple[ImageSplitData, Dict[tuple, int], Dict[tuple, int]]]:
    """
    使用给定的模型对心脏钙化数据进行预测。

    参数:
    model: 用于预测的机器学习模型
    data_processor: HeartCalcificationDataProcessor 实例,包含处理后的数据

    返回:
    List[Tuple[str, List[int], List[int]]]: 预测结果列表,每个元素为 (imageName, true_labels, predicted_labels)
    """

    results = []
    for image_name, image_data in data_processor.data_dict.items():
        images = handle_images(image_data.get_vessel_split_images().values())
        # print(len(images))
        # print(image_name)
        print(f"model :{model}")
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        # print(preds)
        
        pred_dict = {}
        counter = 0
        for key, label in image_data.get_vessel_labels().items():
            pred_dict[key] = preds[counter].item()
            counter += 1
        
        results.append((image_data, image_data.get_vessel_labels(), pred_dict))

    return results


def handle_images(split_images):   
    print(f"1: {split_images}")
    # 將 List[np.ndarray] 轉換為 np.ndarray
    processed_images = np.array([torch.from_numpy(img).float() / 255.0 for img in split_images])  # 將 np.ndarray 轉換為張量
    print(f"1: {processed_images.shape}")
    processed_images = np.expand_dims(processed_images, axis=1) # 增加一個維度，變為 (*, 1, 45, 45)
    print(f"3: {processed_images.shape}")
    processed_images = torch.from_numpy(processed_images)
    print(f"4: {processed_images.shape}")
    
    processed_images = processed_images.repeat(1, 3, 1, 1)  # 從 (*, 1, 45, 45) 擴展為 (*, 3, 45, 45)
    print(f"2: {processed_images.shape}")
    
    return processed_images  # 返回修改完的 split_images

print(model)
data_processor = get_data_processor()
print(model)
results = predict_2(model, data_processor)
print(model)
visualize_predict(results, data_dir, save_dir,  grid_size, resize_height, use_vessel_mask)

None
None
1: dict_values([array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 139, 149, 157],
       [  0,   0,   0, ..., 151, 159, 166],
       [  0,   0,   1, ..., 163, 170, 175]], dtype=uint8), array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [165, 173, 179, ..., 190, 189, 188],
       [173, 178, 182, ..., 188, 188, 189],
       [180, 182, 185, ..., 188, 188, 191]], dtype=uint8), array([[  0,   2,   5, ..., 174, 179, 180],
       [  7,   8,  10, ..., 182, 184, 184],
       [ 16,  15,  16, ..., 188, 187, 186],
       ...,
       [ 14,  19,  31, ...,   6,   5,   8],
       [ 19,  25,  38, ...,   7,   7,  10],
       [ 24,  31,  44, ...,  10,  10,  13]], dtype=uint8), array([[182, 183, 186, ..., 191, 191, 193],
       [183, 184, 186, ..., 194, 195, 196],
       [187, 

TypeError: 'NoneType' object is not callable